In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df_clean.csv'

df_clean = pd.read_csv(url)
df_clean["DATE"] = pd.to_datetime(df_clean["DATE"],dayfirst=True)
for col in df_clean.columns[1:]:
    df_clean[col] = pd.to_numeric(df_clean[col],errors='coerce')

print(df_clean)

           DATE    NCF    RCF    WTI    HH  Brent  NBP_UK  GBP-USD       NBP
0    2009-01-02  74.35  81.00  46.17  5.41  46.91   54.74   1.4453  7.911572
1    2009-01-05  77.65  84.40  48.61  5.83  49.62   56.66   1.4708  8.333553
2    2009-01-06  81.25  88.50  48.56  6.10  50.53   59.88   1.4919  8.933497
3    2009-01-07  78.65  86.25  42.75  5.89  45.86   58.03   1.5111  8.768913
4    2009-01-08  79.15  84.00  41.68  5.96  44.67   55.74   1.5202  8.473595
...         ...    ...    ...    ...   ...    ...     ...      ...       ...
2851 2020-08-25  49.75  48.90  43.17  2.54  45.86   24.61   1.3148  3.235723
2852 2020-08-26  49.70  48.95  43.21  2.52  45.64   25.10   1.3193  3.311443
2853 2020-08-27  49.80  48.90  42.88  2.52  45.09   26.68   1.3206  3.523361
2854 2020-08-28  50.60  52.20  42.96  2.46  45.81   28.95   1.3344  3.863088
2855 2020-08-31  51.95  52.90  42.61  2.30  45.28   28.95   1.3379  3.873220

[2856 rows x 9 columns]


In [2]:
rank = vecm.select_coint_rank(endog=df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF', 'RCF']],
                              k_ar_diff=4,
                              det_order=0)

lag_order1 = vecm.select_order(data=df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF', 'RCF']],
                              maxlags=3, deterministic="ci")

In [3]:
print(lag_order1)
print(rank)

<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 3, BIC -> 2, FPE -> 3, HQIC -> 2>
Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   6          134.1          95.75
  1   6          76.10          69.82
  2   6          50.38          47.85
  3   6          29.03          29.80
-------------------------------------


In [4]:
m1 = vecm.VECM(df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF', 'RCF']], 
               deterministic="ci", k_ar_diff=2, coint_rank=4)
vecm_res_1 = m1.fit()
vecm_res_1.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.WTI,-0.3320,0.025,-13.308,0.000,-0.381,-0.283
L1.HH,0.0170,0.203,0.084,0.933,-0.381,0.415
L1.Brent,0.2385,0.033,7.171,0.000,0.173,0.304
L1.NBP,-0.4298,0.207,-2.079,0.038,-0.835,-0.025
L1.NCF,-0.0052,0.033,-0.158,0.874,-0.069,0.059
L1.RCF,0.0140,0.040,0.349,0.727,-0.065,0.092
L2.WTI,-0.1326,0.025,-5.397,0.000,-0.181,-0.084
L2.HH,-0.1085,0.203,-0.535,0.593,-0.506,0.289
L2.Brent,0.0988,0.032,3.039,0.002,0.035,0.162
L2.NBP,0.1538,0.207,0.741,0.458,-0.253,0.561


In [11]:
granger_results = vecm_res_1.test_granger_causality(caused="NBP", signif=0.05)
granger_results.summary()

Test statistic,Critical value,p-value,df
0.4201,1.667,0.974,"(15, 16962)"


In [6]:
print(vecm_res_1.summary().as_latex())

\begin{center}
\begin{tabular}{lcccccc}
\toprule
                  & \textbf{coef} & \textbf{std err} & \textbf{z} & \textbf{P$> |$z$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midrule
\textbf{L1.WTI}   &      -0.3320  &        0.025     &   -13.308  &         0.000        &       -0.381    &       -0.283     \\
\textbf{L1.HH}    &       0.0170  &        0.203     &     0.084  &         0.933        &       -0.381    &        0.415     \\
\textbf{L1.Brent} &       0.2385  &        0.033     &     7.171  &         0.000        &        0.173    &        0.304     \\
\textbf{L1.NBP}   &      -0.4298  &        0.207     &    -2.079  &         0.038        &       -0.835    &       -0.025     \\
\textbf{L1.NCF}   &      -0.0052  &        0.033     &    -0.158  &         0.874        &       -0.069    &        0.059     \\
\textbf{L1.RCF}   &       0.0140  &        0.040     &     0.349  &         0.727        &       -0.065    &        0.092     \\
\textbf{L2.WTI}   &      -0.1326  &     